<div style="
    background: linear-gradient(135deg, #4A90E2, #007AFF);
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.15);
">
  <h2 style="color:white; text-align:center; font-size:22px; margin:4px 0;">
     Projet : Implémentation
  </h2>
</div>

<h3 style="color:#333; font-size:18px; margin-top:16px;">Début du code</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 14px 20px;
    border-radius: 8px;
    margin-top: 10px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
La première chose implémentée dans notre code est <strong>l’importation des bibliothèques essentielles</strong> au bon fonctionnement du programme.  
<br><br>
La bibliothèque <code>csv</code> est utilisée pour la lecture et l’exploitation des fichiers .csv, permettant de charger facilement les données d’entrée.  
<br>
<code>NumPy</code> est employée afin de réaliser des opérations mathématiques et logiques de manière efficace, notamment sur des tableaux et matrices.  
<br>
Enfin, la structure <code>deque</code> est utilisée comme une file doublement chaînée(double-ended queue), offrant des opérations rapides d’ajout et de suppression d’éléments aux deux extrémités.  
<br><br>
Ces importations constituent la base de l’implémentation, préparant l’environnement avant la mise en place des fonctions principales.
</div>


In [ ]:
import csv
import numpy as np
from collections import deque
import copy
import random
import time
import math
import os
import matplotlib.pyplot as plt

<h3 style="color:#333; font-size:18px; margin-top:16px;">Gestion des instances et paramètres</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 14px 20px;
    border-radius: 8px;
    margin-top: 10px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">Le switch nous permet de choisir quel instance nous voulons utiliser pour notre lancement, à l'aide d'un input on rentre l'instance de notre choix ainsi que le nombre de camion que l'on veut, sachant que si on met plus de camion qu'il n'y a de client le code tournera en boucle, il faudra alors l'arreter. Il y a aussi des constantes comme la durée max d'un cycle, le changement d'heure passant de 8h à 12h à 16h, cela entraine un changement de matrice pour simuler les bouchons.
</div>

In [ ]:
# ================== CONFIG / SWITCH ==================
print("=== Sélection de l'instance ===")
print("1 - 6X6.csv")
print("2 - 11X11.csv")
print("3 - 51X51.csv")
print("4 - 101X101.csv")
print("5 - 201X201.csv")
print("6 - 501X501.csv")
print("7 - 1001X1001.csv")
print("8 - 1501X1501.csv")
print("9 - 2001X2001.csv")
choix = input("Choisissez la matrice à utiliser (1-9) : ")

if choix == "1":
    csv_path = "instance/6X6.csv"
    nameFile = "6X6.csv"
elif choix == "2":
    csv_path = "instance/11X11.csv"
    nameFile = "11X11.csv"
elif choix == "3":
    csv_path = "instance/51X51.csv"
    nameFile = "51X51.csv"
elif choix == "4":
    csv_path = "instance/101X101.csv"
    nameFile = "101X101.csv"
elif choix == "5":
    csv_path = "instance/201X201.csv"
    nameFile = "201X201.csv"
elif choix == "6":
    csv_path = "instance/501X501.csv"
    nameFile = "501X501.csv"
elif choix == "7":
    csv_path = "instance/1001X1001.csv"
    nameFile = "1001X1001.csv"
elif choix == "8":
    csv_path = "instance/1501X1501.csv"
    nameFile = "1501X1501.csv"
elif choix == "9":
    csv_path = "instance/2001X2001.csv"
    nameFile = "2001X2001.csv"
else:
    csv_path = "instance/6X6.csv"
    nameFile = "6X6.csv"

try:
    nbTrucks = int(input("Nombre de camions à utiliser : "))
except:
    nbTrucks = 10


depot = 0
MAX_CYCLE_TIME = 720  
SEUIL1 = 240
SEUIL2 = 480

<h3 style="color:#333; font-size:18px; margin-top:16px;">Lecture des fichiers CSV</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 14px 20px;
    border-radius: 8px;
    margin-top: 10px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
Cette partie du programme est dédiée à la lecture des fichiers CSV.  
<br><br>
La fonction <code>lire_matrice_csv</code> ouvre un fichier CSV (généralement celui indiqué en entrée), puis :  
<ul style="margin-top:6px; margin-bottom:6px;">
  <li>Elle lit le fichier ligne par ligne.</li>
  <li>Elle ignore les espaces inutiles et convertit toutes les valeurs en entiers.</li>
  <li>Elle stocke chaque ligne dans une matrice sous forme de liste de listes.</li>
</ul>
Avant de lire le fichier, le programme vérifie si le dossier “matrice” existe ; s’il n’existe pas, il est créé automatiquement.  
<br>
Ensuite, on tente de lire le fichier avec <code>NumPy</code>. Si le fichier contient des espaces, la fonction <code>lire_matrice_csv</code> est utilisée pour assurer une lecture correcte.  
<br>
Enfin, si le fichier est introuvable, une erreur est générée pour informer l’utilisateur.
</div>


In [ ]:
# === Utilitaires de lecture (définit avant usage) ===
def lire_matrice_csv(filename):
    matrice = []
    with open(filename, newline='') as f:
        lecteur = csv.reader(f)
        for ligne in lecteur:
            # ignorer champs vides
            valeurs = [int(float(x)) for x in ligne if x.strip() != ""]
            if valeurs:
                matrice.append(valeurs)
    return matrice


if not os.path.exists("matrice"):
    os.makedirs("matrice")

# === Chargement de la matrice de base (celle choisie) ===
try:

    try:
        base_matrix = np.loadtxt(csv_path, delimiter=",", dtype=int).tolist()
    except Exception:
        base_matrix = lire_matrice_csv(csv_path)
except FileNotFoundError:
    raise FileNotFoundError(f"Fichier introuvable : {csv_path}")

matrix = base_matrix

<h3 style="color:#333; font-size:18px; margin-top:16px;">Fonctions principales : voisinMinPoid & poidCycle</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 14px 20px;
    border-radius: 8px;
    margin-top: 10px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
La fonction voisinMinPoid va chercher le voisin le plus proche pour limiter les couts ainsi que les émissions. Pour cela nous utilisons 3 paramètres, matrice de distances, la liste des clients encore disponibles et l'index du client actuel. Pour chaque client i dans la liste, on verifie s'il existe bien une route, s'il est le premier client du camion ou qu'il est plus proche que le meilleur alors il le remplace. La fonction retourne l'index du client et la distance pour l'atteindre.

poidCycle calcule le poids total des trajets de tous les camions.

</div>

In [ ]:
# ================== Fonctions algorithme (inchangées en interface) ==================
def voisinMinPoid(matrix_local, listeClient, cur):
    poidMinTrajet = 0
    nextVoisin = -1
    for i in listeClient:
        if matrix_local[cur][i] > 0 and (poidMinTrajet == 0 or matrix_local[cur][i] < poidMinTrajet):
            nextVoisin = i
            poidMinTrajet = matrix_local[cur][i]
    return nextVoisin, poidMinTrajet

def poidCycle():
    return sum(truckCycles[0])

<h3 style="color:#333; font-size:18px; margin-top:16px;">Recherche Tabou pour les cycles</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 14px 20px;
    border-radius: 8px;
    margin-top: 10px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
La fonction <code>recherche_tabou_cycle</code> permet d’effectuer une recherche à l’aide de la méthode tabou afin de trouver un cycle optimal dans la matrice.  
<br><br>
La méthode tabou est une métaheuristique d’optimisation dont le principe consiste à explorer plusieurs solutions en choisissant, à chaque itération, le voisin qui minimise le mieux la fonction objectif. Ce voisin est ensuite ajouté à une liste tabou, pour éviter qu’il ne soit réévalué lors des itérations suivantes.  
<br><br>
Le processus fonctionne de la manière suivante :  
<ul style="margin-top:6px; margin-bottom:6px;">
  <li>La liste tabou est initialisée.</li>
  <li>On identifie le camion ayant actuellement le trajet le plus court afin de le faire avancer vers son voisin le plus favorable.</li>
  <li>Ce voisin est ajouté à la liste tabou.</li>
  <li>Le processus est répété jusqu’à ce que tous les sommets de la matrice soient présents dans la liste tabou.</li>
</ul>
Enfin, on ajoute à chaque camion le temps nécessaire pour retourner au dépôt, afin de compléter son cycle et obtenir la solution finale.
</div>


In [ ]:
def recherche_tabou_cycle(matrix, start):
    tabou = deque(maxlen=len(matrix))
    tabou.append(start)
    ancientCandidats = list(range(len(matrix)))
    
    for i in range(nbTrucks):
        tabou.append(truckCycles[1][i][-1])

    while len(tabou) < len(matrix):

        truckAtMove = truckCycles[0].index(min(truckCycles[0]))
        cur = truckCycles[1][truckAtMove][-1]

        if  truckCycles[0][truckAtMove] < SEUIL1:
            matrixAtUse = matrix8h
        elif truckCycles[0][truckAtMove] < SEUIL2:
            matrixAtUse = matrix12h
        elif truckCycles[0][truckAtMove] >= MAX_CYCLE_TIME:
            print(f"Camion {truckAtMove} n'a plus de temps pour terminer sa tourné. Essayez avec plus de camions.")
            break
        else:
            matrixAtUse = matrix16h
        
        
        candidats = [i for i in ancientCandidats if i not in tabou]
        ancientCandidats = candidats


        if not candidats:
            break

        voisin, temps = voisinMinPoid(matrixAtUse, candidats, cur)

        truckCycles[1][truckAtMove].append(voisin)
        truckCycles[0][truckAtMove] += temps
        tabou.append(voisin)

    # Retour au dépôt
    for i in range(nbTrucks):
        last_visited = truckCycles[1][i][-1]
        truckCycles[1][i].append(depot)
        truckCycles[0][i] += matrix[last_visited][depot]

<h3 style="color:#333; font-size:18px; margin-top:16px;">Exécution multiple de l’algorithme tabou</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 14px 20px;
    border-radius: 8px;
    margin-top: 10px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
Cette fonction permet d’exécuter plusieurs lancements de l’algorithme tabou afin de trouver le meilleur itinéraire possible pour les camions.  
<br><br>
Trois paramètres principaux sont définis :  
<ul style="margin-top:6px; margin-bottom:6px;">
  <li><code>tempsMeilleurCycle</code> : stocke le meilleur temps, initialisé à l’infini afin que le premier temps calculé devienne le meilleur.</li>
  <li><code>goodI</code> : numéro du meilleur lancement.</li>
  <li><code>bestTime</code> : sauvegarde le meilleur ensemble de cycles trouvé.</li>
</ul>
Avant de démarrer les lancements, le programme vérifie que les matrices simulant les bouchons sont bien créées dans le dossier “matrice”, afin de refléter les conditions de circulation à différentes heures de la journée.  
<br>
La première boucle <code>for i in range(nb_lancement)</code> sert à exécuter indépendamment chaque lancement, en attribuant un **point de départ aléatoire à chaque camion.  
<br>
Pour chaque lancement :  
<ul style="margin-top:6px; margin-bottom:6px;">
  <li>On initialise la structure <code>truckCycles</code> qui contient la durée totale du trajet et la liste des clients visités.</li>
  <li>La boucle <code>for j in range(nbTrucks)</code> fait partir chaque camion du dépôt (0), lui assigne un premier client aléatoire différent des autres camions, et calcule la distance du premier trajet.</li>
</ul>
Cette approche permet de tester plusieurs configurations initiales et d’augmenter les chances de trouver l’itinéraire le plus optimal.
</div>


In [ ]:
def tabou_multi_start(matrix_local, nb_lancements=20):
    tempsMeilleurCycle = float('inf')
    goodI = -1
    bestTime = None

    try:
        creer_fichiers_avec_bouchons()
    except Exception as e:
        print(f"Création fichiers bouchons échouée ou déjà faite : {e}")

    for i in range(nb_lancements):
        global truckCycles
        truckCycles = [
            [0] * nbTrucks,
            [[] for _ in range(nbTrucks)]
        ]

        for j in range(nbTrucks):
            truckCycles[1][j] = [depot]
            attempts = 0
            while True:
                attempts += 1
                if attempts > 1000:
                    raise RuntimeError("Impossible d'initialiser firstNeighbor (trop d'essais).")
                firstNeighbor = random.randint(1, len(matrix_local)-1)
                if not any(firstNeighbor in cycle for cycle in truckCycles[1]):
                    truckCycles[1][j].append(firstNeighbor)
                    truckCycles[0][j] = matrix_local[depot][firstNeighbor]
                    break

        recherche_tabou_cycle(matrix_local, depot)

        total = poidCycle()
        print(f"Lancement {i+1} terminé : Temps du cycle = {total}")

        for k in range(nbTrucks):
            print(f"Premier client du camion {k+1} : {truckCycles[1][k][0]+1}")
            print(f"Cycle du camion {k+1} : ", " -> ".join(str(x+1) for x in truckCycles[1][k]))
            print(f"Temps total du camion {k+1} : {truckCycles[0][k]}")
            print()

        if total < tempsMeilleurCycle:
            tempsMeilleurCycle = total
            goodI = i
            bestTime = [
                truckCycles[0].copy(),
                [cycle.copy() for cycle in truckCycles[1]]
            ]
            print(f"→ Nouveau meilleur cycle sauvegardé ! Lancement {i+1}.\n")

    return tempsMeilleurCycle, goodI, bestTime

<h3 style="color:#333; font-size:18px; margin-top:16px;">Simulation du trafic routier</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 14px 20px;
    border-radius: 8px;
    margin-top: 10px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
Ces fonctions permettent de **simuler le trafic routier et ses variations** dans la matrice des distances.  
<br><br>
<ul style="margin-top:6px; margin-bottom:6px;">
  <li><code>generer_facteur_bouchon</code> : calcule un facteur de congestion en fonction de l’heure, augmentant les temps de trajet lors des heures de pointe (par exemple à <strong>8h</strong> et <strong>16h</strong>).</li>
  <li><code>facteurs_variation</code> : sélectionne un pourcentage de routes aléatoires et modifie légèrement leur valeur pour représenter des perturbations locales (travaux, accidents, conditions variables).</li>
  <li><code>creer_fichiers_avec_bouchons</code> : combine ces deux mécanismes pour générer trois nouvelles matrices correspondant aux conditions de circulation à 8h, 12h et 16h.</li>
</ul>
Ces matrices sont ensuite utilisées par l’algorithme principal pour adapter les trajets des camions en fonction du trafic, garantissant une simulation plus réaliste et des itinéraires optimisés.
</div>


In [ ]:
# ================== Partie bouchons (création des 3 matrices horaires) ==================
def generer_facteur_bouchon(heure):
    seed_value = hash(f"bouchon_{heure}") % (2**32)
    random.seed(seed_value)
    intensite = 0.5 + 0.5 * math.sin((heure - 8) / 24 * 2 * math.pi)
    facteur = 2.0 * intensite
    if facteur <= 0:
        facteur = 1
    return facteur

def facteurs_variation(matrice, pourcentage):
    n = len(matrice)
    toutes_les_routes = [(i, j) for i in range(n) for j in range(i + 1, n) if matrice[i][j] != 0]
    nb_a_modifier = int(len(toutes_les_routes) * pourcentage)
    if nb_a_modifier <= 0:
        return []
    routes_selectionnees = random.sample(toutes_les_routes, nb_a_modifier)
    for i, j in routes_selectionnees:
        p = random.uniform(-0.3, 0.3)
        nouvelle_valeur = matrice[i][j] * (1 + p)
        matrice[i][j] = matrice[j][i] = max(1, int(round(nouvelle_valeur)))
    return routes_selectionnees

def creer_fichiers_avec_bouchons():

    heures = [8, 12, 16]
    matrixCreated = []
    matrice_copie = copy.deepcopy(matrix)

 
    chemin_original = csv_path  
    try:
        matrice_base = lire_matrice_csv(chemin_original)
    except FileNotFoundError:
        print(f"Fichier source introuvable : {chemin_original}")
    

    n = len(matrice_base)
    base_name = os.path.basename(csv_path).replace('.csv','')

    for heure in heures:
        nom_sortie = f"matrice/{base_name}_{heure}h.csv"
        facteur_global = generer_facteur_bouchon(heure)

        proportion_routes_affectees = 0.3
        routes_affectees = set()
        for i in range(n):
            for j in range(i + 1, n):
                if random.random() < proportion_routes_affectees:
                    routes_affectees.add((i, j))

        for i in range(n):
            for j in range(i + 1, n):
                if (i, j) in routes_affectees:
                    variation_locale = random.uniform(0.8, 1.4)
                    facteur_total = facteur_global * variation_locale
                    nouvelle_valeur = int(round(matrice_base[i][j] * facteur_total))
                    matrice_copie[i][j] = matrice_copie[j][i] = nouvelle_valeur
                else:
                    matrice_copie[i][j] = matrice_copie[j][i] = int(matrice_base[i][j])

        # sauvegarde (écrase si existant)
        with open(nom_sortie, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(matrice_copie)

        print(f"✓ Fichier créé : {nom_sortie}")
        matrixCreated.append((nom_sortie))
    
    matrix8h = lire_matrice_csv(matrixCreated[0])
    matrix12h = lire_matrice_csv(matrixCreated[1])
    matrix16h = lire_matrice_csv(matrixCreated[2])

    return matrix8h, matrix12h, matrix16h

<h3 style="color:#333; font-size:18px; margin-top:16px;">Simulation et vérification du trafic</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 14px 20px;
    border-radius: 8px;
    margin-top: 10px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
Ces fonctions permettent de **simuler et vérifier le comportement du trafic routier** dans les matrices générées.  
<br><br>
<ul style="margin-top:6px; margin-bottom:6px;">
  <li><code>cout_effectif</code> : calcule le coût réel d’un trajet entre deux points <code>i</code> et <code>j</code> à une heure donnée. Elle applique le facteur de bouchon correspondant pour simuler l’augmentation du temps de trajet due au trafic.</li>
  <li><code>simulation_journee</code> : permet de tester la variation du trafic au cours d’une journée. Elle affiche toutes les 4 heures le facteur de congestion et le coût du trajet entre deux points (ici 0 et 5) afin d’observer l’évolution des conditions de circulation.</li>
  <li><code>verifier_modifications</code> : contrôle la cohérence des matrices modifiées. Elle compare la matrice originale avec ses versions modifiées (8h, 12h, 16h) pour vérifier que le bon pourcentage de routes a bien été modifié, garantissant la validité de la génération des bouchons.</li>
</ul>
Ces fonctions assurent ainsi que les simulations de trafic sont réalistes et fiables, permettant à l’algorithme principal d’adapter les trajets des camions de manière optimale.
</div>


In [ ]:
# ================== Fonctions de simulation / affichage (inchangées) ==================
def cout_effectif(matrice_local, i, j, heure):
    base = matrice_local[i][j]
    if base == 0:
        return 0
    facteur_bouchon = generer_facteur_bouchon(heure)
    cout = base * facteur_bouchon
    return max(1, int(round(cout, 0)))

def simulation_journee(matrice_local, nom_fichier):
    print(f"\n=== Simulation sur {nom_fichier} ===")
    heures = list(range(0, 25, 4))
    for h in heures:
        facteur = generer_facteur_bouchon(h)
        cout_05 = cout_effectif(matrice_local, 0, 5, h)
        print(f"Heure {h:2d}h | Facteur bouchon: {facteur:.2f} | Coût 0->5: {cout_05}")

# ================== Vérifications et tests ==================
def verifier_modifications():
    print("🔍 VÉRIFICATION DES MODIFICATIONS")
    print("=" * 50)
    random.seed(42)

    try:
        matrice_test = lire_matrice_csv(csv_path)
    except FileNotFoundError:
        print(f"Fichier introuvable : {csv_path}")
        return

    n = len(matrice_test)
    routes_non_nulles_original = sum(1 for i in range(n) for j in range(i+1,n) if matrice_test[i][j] != 0)
    print(f"Routes non-nulles originales: {routes_non_nulles_original}")

    for heure in [8, 12, 16]:
        print(f"\n--- Heure {heure}h ---")
        matrice_copie = copy.deepcopy(matrice_test)
        random.seed(hash(f"test_{heure}") % (2**32))
        modifications = facteurs_variation(matrice_copie, 0.3)
        routes_modifiees = sum(1 for i in range(n) for j in range(i+1,n) if matrice_copie[i][j] != matrice_test[i][j])
        print(f"Routes modifiées comptées: {routes_modifiees}")
        print(f"Modifications annoncées: {len(modifications)}")
        print(f"COHÉRENT: {routes_modifiees == len(modifications)}")

<h3 style="color:#333; font-size:18px; margin-top:16px;">Bloc principal d’exécution</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 14px 20px;
    border-radius: 8px;
    margin-top: 10px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
Cette partie correspond au bloc principal d’exécution du programme.  
<br><br>
Lorsque le script est lancé, plusieurs étapes sont effectuées automatiquement :  
<ol style="margin-top:6px; margin-bottom:6px;">
  <li><strong>Création des matrices avec bouchons</strong> grâce à la fonction <code>creer_fichiers_avec_bouchons()</code>, générant trois matrices représentant les conditions de circulation à <strong>8h, 12h et 16h</strong>.</li>
  <li><strong>Vérification de la cohérence des modifications</strong> avec <code>verifier_modifications()</code>, afin de s’assurer que les matrices ont bien été modifiées comme prévu.</li>
  <li><strong>Exécution de l’algorithme principal</strong> avec <code>tabou_multi_start(matrix)</code>, qui cherche à trouver le meilleur ensemble de trajets (cycles) pour les camions en tenant compte des contraintes de temps et du trafic.</li>
  <li><strong>Affichage du meilleur résultat</strong> une fois la recherche terminée :  
    <ul style="margin-top:4px; margin-bottom:4px;">
      <li>Le numéro du lancement ayant produit le meilleur cycle</li>
      <li>Le détail du trajet de chaque camion</li>
      <li>Le temps total d’exécution de la simulation</li>
    </ul>
  </li>
</ol>
En résumé, ce bloc permet de lancer la génération, la vérification et la simulation complète du système de livraison, en tenant compte du trafic routier simulé.
</div>


In [ ]:
# ================== MAIN : exécution ==================
if __name__ == "__main__":
    matrix8h, matrix12h, matrix16h = creer_fichiers_avec_bouchons()

    verifier_modifications()

    start_time = time.time()
    tempsMeilleurCycle, goodI, bestTime = tabou_multi_start(matrix)
    execution_time_ms = (time.time() - start_time) * 1000

    print("\n=== Meilleur cycle trouvé ===")
    print("Lancement n°", goodI+1, "  Temps du cycle :", tempsMeilleurCycle)
    if bestTime:
        for i in range(nbTrucks):
            print(f"Cycle du camion {i+1} : ", " -> ".join(str(x+1) for x in bestTime[1][i]))
            print(f"Temps total du camion {i+1} : {bestTime[0][i]}\n")
    else:
        print("Aucun meilleur cycle sauvegardé.")

    print("Temps d'exécution :", round(execution_time_ms, 2), "ms")